In [39]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import os

In [40]:
def get_csv_data(path_logs, path_csv):
    data = pd.read_csv(path_csv, sep='\t')
    dfs = []
    for dirname, _, filenames in os.walk(path_logs):
        for filename in filenames:
            if filename.endswith('.csv'):
                file_path = os.path.join(dirname, filename).replace('\\', '/')
                df = pd.read_csv(file_path,sep=' ')
                
                row = data.loc[data['log_id'] == int(filename[:-4])] # https://stackoverflow.com/questions/17071871/how-do-i-select-rows-from-a-dataframe-based-on-column-values
                # print(row)
                arr = row.values.tolist()
                arr.append(df)
                dfs.append(arr)
    return dfs

In [41]:
dfs = get_csv_data('../the-attentive-cursor-dataset/logs', '../the-attentive-cursor-dataset/groundtruth.tsv')

In [42]:
cleandf = [] #note + liste des inputs
for row in range(len(dfs)):
    cleandf.append([dfs[row][0][2],dfs[row][1]['event'].value_counts()])

In [43]:
tab = np.array(cleandf, dtype='object')
for row in tab :
    for input in ['mouseup','focus','load','beforeunload','unload','touchend'] : #liste arbitraire des actions à enlever
        if input in row[1].keys() :
            row[1].pop(input)

#liste des actions à conserver
all_inputs = []
for row in tab :
    for input in row[1].keys() :
        if input not in all_inputs :
            all_inputs.append(input)

In [46]:
tmp_all_x = [] #stock temporairement les données d'entrées
tmp_all_y = [] #stock temporairement les données de sortie

for row in tab :
    tmp_all_x.append([])
    for input in all_inputs :
        if input in row[1].keys() :
            tmp_all_x[len(tmp_all_x)-1].append(row[1].get(input))
        else :
            tmp_all_x[len(tmp_all_x)-1].append(0)
    tmp_all_y.append(row[0])

all_x = np.array(tmp_all_x) #données en entrée du modèle
all_y = np.array(tmp_all_y) #données en sortie du modèle
print(all_x)

[[61 22  9 ...  0  0  0]
 [ 7  9  3 ...  0  0  0]
 [11 39  3 ...  0  0  0]
 ...
 [ 0  8  4 ...  0  0  0]
 [ 0  8  2 ...  0  0  0]
 [ 0  4  2 ...  0  0  0]]


In [45]:
inputs = keras.Input(shape=(len(all_inputs),))
x = tf.keras.layers.Dense(4, activation=tf.nn.relu)(inputs)
outputs = tf.keras.layers.Dense(6, activation=tf.nn.softmax)(x)

model = keras.Model(inputs=inputs, outputs = outputs)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

history = model.fit(all_x, all_y, batch_size=5, epochs=20, validation_split=0.1)

Epoch 1/20
279/279 [==============================] - 3s 5ms/step - loss: 1.6825 - accuracy: 0.3683 - val_loss: 1.6656 - val_accuracy: 0.3548
Epoch 2/20
279/279 [==============================] - 1s 3ms/step - loss: 1.6659 - accuracy: 0.3640 - val_loss: 1.6481 - val_accuracy: 0.3548
Epoch 3/20
279/279 [==============================] - 1s 4ms/step - loss: 1.6612 - accuracy: 0.3640 - val_loss: 1.6468 - val_accuracy: 0.3548
Epoch 4/20
279/279 [==============================] - 1s 3ms/step - loss: 1.6574 - accuracy: 0.3690 - val_loss: 1.6454 - val_accuracy: 0.3484
Epoch 5/20
279/279 [==============================] - 1s 3ms/step - loss: 1.6554 - accuracy: 0.3597 - val_loss: 1.6392 - val_accuracy: 0.3806
Epoch 6/20
279/279 [==============================] - 1s 3ms/step - loss: 1.6538 - accuracy: 0.3618 - val_loss: 1.6383 - val_accuracy: 0.3742
Epoch 7/20
279/279 [==============================] - 1s 3ms/step - loss: 1.6531 - accuracy: 0.3640 - val_loss: 1.6373 - val_accuracy: 0.3742
Epoch 